## Ice Temperature (icetemp) Package Tutorial

In [1]:
# Standard imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Package for ice temperature estimation
import icetemp

This package is intended to infer the change in temperature from measurements of temperature ($^\circ$C) and depth (m) off of thermistor provided by the AMANDA and IceCube collaborations. Our baseline hypothesis determines a linear relationship between the ice temperature and the depth. We contruct a generative model used to analyze the data involves an uniform prior and normal likelihood.

$$ \mu_T \sim \text{uniform}(0.01^{\circ}\text{C},0.1^{\circ}\text{C})$$
$$ \Delta T \sim \text{normal}(\mu_T,\sigma_\text{noise})$$
$$ \mathcal{L} \propto \prod \frac{}{\sqrt{}} * exp() $$

Noise sources considered are directly related to the thermistor technical information. 

The calculatations of the likelihood function from your statistical model is given by: 

In [5]:
# load some data
# FIXME: change to load the example data
from data_io import load_ice_data
ex_data = load_ice_data("icecube_temp.2007.txt", data_year=2007,temp_errors=0.1, depth_errors=2)
print(ex_data['Temperature'])

0    -28.76
1    -25.09
2    -20.93
3    -26.51
4    -22.42
5    -17.89
6    -25.79
7    -21.55
8    -18.23
9    -26.26
10   -17.69
Name: Temperature, dtype: float64


In [6]:
#likelihood = icetemp.likehood()